In [ ]:
#!!!АЛГОРИТМ РАБОТАЕТ ТОЛЬКО ПОШАГОВО!!!
#pip install haversine
import pandas as pd
import numpy as np
from haversine import haversine, Unit
import ipywidgets as widgets
import sys
from io import StringIO
from IPython import get_ipython
from IPython.display import Image
from ipyleaflet import Map, Marker, basemaps, basemap_to_tiles
from ipywidgets import Layout

In [ ]:
#Определение географических координат
center = (47.03823025939306, 28.852018195122298)
m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=7)
marker = Marker(location=center, draggable=True)
m.add_layer(marker)
display(m)
def handle_click(**kwargs):
    print('Широта, долгота: ', kwargs['coordinates'])
marker.on_click(handle_click)

#ЧТОБЫ УЗНАТЬ КООРДИНАТЫ, ПЕРЕМЕСТИТЕ МАРКЕР НА НУЖНУЮ ТОЧКУ И НАЖМИТЕ НА НЕГО

In [ ]:
#Выбор Ветрогенератора
print('Выбор ветрогенератора')
Wind_Turbines = pd.read_excel('Wind_Turbines.xlsx')
WT_Models = list(Wind_Turbines.columns)
del WT_Models[0]
Select_WT = widgets.Select(
    options=(WT_Models),
    value=('Aeolos-H 10kW, 12 m'),
    description='',
    disabled=False
)
Select_WT

In [ ]:
#Задай координаты точки в десятичном виде (lat, lon) 
X1 = float(input('Задай широту в десятичном виде:\n'))
X2 = float(input('Задай долготу в десятичном виде:\n'))
X = (X1, X2) #(47.03823025939306, 28.852018195122298)

In [ ]:
#ТАБЛИЦА МИЛЕВСКОГО
PATH = "/Users/Legion/"
Image(filename = PATH + "Milevski_Table.jpg", width=827, height=668)

In [ ]:
#Определение классов открытости по Милевскому для площадки ВЭС
MX = [0] * 8
C_D = ['Север', 'Северо-Восток', 
       'Восток', 'Юго-Восток', 'Юг', 
       'Юго-Запад', 'Запад', 'Северо-Запад']
for i in range(len(C_D)):
    print('Задай коэффициент по таблице Милевского для направления:', C_D[i])
    i = int(i)
    i = i - 1
    MX[i] = int(input())

In [ ]:
#Импорт список Метеостанций
Meteostations = pd.read_csv('Meteostations.txt', sep=",", header=None)
Meteostations.columns = ["Meteostation", "latitude", "longitude"]

#Расчет расстояния от точки Х до каждой из Метеостанций
Rez=()
lis=()
i=0
while i<=len(Meteostations.index)-1:
    Meteo = (Meteostations.iloc[i]['latitude'], 
             Meteostations.iloc[i]['longitude']) #получить вид (lat, lon)
    Rez = round(haversine(X, Meteo),3)
    lis = np.append([lis],[Rez])
    i = i+1
    
#Выбор ближайшей метеостанции
numb = np.where(lis == min(lis))

#Импорт данных выбранной метеостанции
Meteostation_X = pd.read_excel((Meteostations.iloc[int(numb[0])]['Meteostation'] + '.xlsx'))
    
#Импорт данных по классу открытости из БД Флюгер
Fluger = pd.read_excel('Fluger.xlsx')
    
#Импорт данных для выбранной метеостанции
Fluger_X = Fluger.loc[numb].drop(['Meteostation'], axis=1)

#Расчет оэффициента k0 для пересчета скорости ветра к условиям площадки ВЭС
k0=()
if int(Fluger_X.loc[numb, 'N']) != 1:
    k0 = round((MX / Fluger_X), 3)
else:
    k0 = Fluger_X
k0

#Пересчет скорости ветра расчетного года к условиям площадки ВЭС
j=0
V_VES=()
while j<len(Meteostation_X):
    loc1 = Meteostation_X.loc[j]['DD']
    loc2 = Meteostation_X.loc[j]['ff']
    if (loc1 >= 0.0 and loc1 < 22.5) or (loc1 >= 337.5 and loc1 <= 360):
        V_VES = np.append([V_VES],[(loc2*k0['N'])])
    elif (loc1 >= 22.5 and loc1 < 67.5):
        V_VES = np.append([V_VES],[(loc2*k0['NE'])])
    elif (loc1 >= 67.5 and loc1 < 112.5):
        V_VES = np.append([V_VES],[(loc2*k0['E'])])
    elif (loc1 >= 112.5 and loc1 < 157.5):
        V_VES = np.append([V_VES],[(loc2*k0['SE'])])
    elif (loc1 >= 157.5 and loc1 < 202.5):
        V_VES = np.append([V_VES],[(loc2*k0['S'])])
    elif (loc1 >= 202.5 and loc1 < 247.5):
        V_VES = np.append([V_VES],[(loc2*k0['SW'])])
    elif (loc1 >= 247.5 and loc1 < 292.5):
        V_VES = np.append([V_VES],[(loc2*k0['W'])])
    elif (loc1 >= 292.5 and loc1 < 337.5):
        V_VES = np.append([V_VES],[(loc2*k0['NW'])])
    elif (loc1 < 0):
        V_VES = np.append([V_VES],[0])
    j = j+1
VV_VES = pd.DataFrame(data=V_VES, columns=['V_VES'])  
#Все в одну таблицу для расчета
Meteostation_XX = Meteostation_X.join(VV_VES)

#Расчет среднемесячных скоростей ветра за расчетный год
def V_Month_Calc(ind):
    V_Month = sum(Meteostation_XX.loc[Meteostation_XX['Month'] == ind]
                  ['V_VES'])/len(Meteostation_XX.loc[Meteostation_XX['Month'] == ind]['V_VES'])
    return V_Month
VV_Month = ()
for i in range(1, 13):
    VV_Month = np.append([VV_Month], [V_Month_Calc(i)])
VV_Month = round(pd.DataFrame(data=VV_Month, columns=['V_Month']), 3)
    
#Импорт коэффицентов для расчета показателя степени m(V) для каждой Метеостанции
Koef_m = pd.read_excel('Stepeni_m.xlsx')

#Расчет коэффициентов m(V) для площадки ВЭС
m_VES = ()
a = Koef_m.loc[numb].drop(['Meteostation'], axis=1)['a']
b = Koef_m.loc[numb].drop(['Meteostation'], axis=1)['b']
for i in range (0, 12):
    m_VES = np.append([m_VES], [a * float(VV_Month.iloc[i]) ** b])
m_VES = round(pd.DataFrame(data = m_VES, columns=['m_VES']), 2)

#Присоединение показателей степени для каждого измерения к таблице с данными расчета
mm_VES = ()
for j in range (1, 13):
    for i in range(len(Meteostation_XX.loc[Meteostation_XX['Month'] == j]['V_VES'])):
        mm_VES = np.append([mm_VES], [m_VES.loc[j-1]])
mm_VES = round(pd.DataFrame(data = mm_VES, columns=['mm_VES']), 2)
Meteostation_XX = Meteostation_XX.join(mm_VES)

#Пересчет скорости ветра на высоту башни ВЭУ на площадке ВЭС
V_VES_H = ()
for i in range(len(Meteostation_XX)):
    V_VES_H = np.append([V_VES_H], 
                        [Meteostation_XX['V_VES'][i]*(float(Wind_Turbines[Select_WT.value][50])/10)
                         **Meteostation_XX['mm_VES'][i]])
V_VES_H = round(pd.DataFrame(data = V_VES_H, columns=['V_VES_H']), 2)

#Расчет первичной выработки ВЭС
WT_Power = ()
xp = Wind_Turbines['V, m/s'][1:50]
fp = Wind_Turbines[Select_WT.value][1:50]
for i in range(len(V_VES_H)):
    if (float(V_VES_H['V_VES_H'][i])) < Wind_Turbines[Select_WT.value][51]:
        WT_Power = np.append([WT_Power], 0)
    else:
        WT_Power = np.append([WT_Power], [np.interp(float(V_VES_H['V_VES_H'][i]), xp, fp)])
WT_Power = round(pd.DataFrame(data = WT_Power, columns=['WT_Power']), 2)
Meteostation_XX = Meteostation_XX.join(WT_Power)

#Расчет финальной выработки ВЭС
E_WT_prim = WT_Power.sum()*8760/len(WT_Power)
Betta = 0.885
E_WT_fin = Betta * E_WT_prim
print('Выработка 1 ВЭУ', Select_WT.value, 'составит', float(round(E_WT_fin, 2)), 'кВт*ч за год')